[ Stefano Pigozzi | Tema Data Analytics | Big Data Analytics | A.A. 2022/2023 | Unimore ]

# Attività online correlata all'avanzare della pandemia COVID

> ### Data processing e exploratory data analytics su dataset provenienti da più sorgenti
> 
> L’attività da svolgere consiste nel:
> 1. Scegliere due o più dataset provenienti da due o più sorgenti.  
>     * Il dataset finale deve essere costituito almeno da due file.
> 2. Usando [pandas](https://pandas.pydata.org/) implementare le operazioni di data processing necessarie (principalmente join e selezioni) per mettere in collegamento i dataset e per preparare i dati al passo successivo
> 3. Usando pacchetti Python quali [pandas](https://pandas.pydata.org/), [scipy](https://scipy.org/), [matplotlib](https://matplotlib.org/) e [seaborn]https://seaborn.pydata.org/) implementare attività di data cleaning, exploratory data analysis estraendo dati statistici e di visualizzazione dei risultati attraverso il quale sia possibile "raccontare qualcosa sui dati" (storytelling), eventualmente partendo da dei quesiti di ricerca. L'uso
dei pacchetti non deve necessariamente essere limitato alle istruzioni viste a lezione. Le documentazioni dei pacchetti stessi e i volumi messi a disposizione su Dolly fornisco spunti d’uso interessanti!
> 4. Produrre un notebook Jupyter (https://jupyter.org/) che contenga:
>     * una introduzione all’argomento scelto, alle sorgenti dati e agli obiettivi del progetto specificando
eventualmente i quesiti di ricerca
>     * una sezione per ogni fase del progetto di data analytics

## Abstract

<!-- TODO -->

## Installazione requisiti

Per funzionare, questo progetto necessita di alcuni package Python, scaricabili da [PyPI](https://pypi.org/).

In particolare:

* [numpy](https://pypi.org/project/numpy/1.23.5/)
* [scipy](https://pypi.org/project/scipy/1.9.3/)
* [pandas](https://pypi.org/project/pandas/1.5.2/)
* [matplotlib](https://pypi.org/project/matplotlib/3.6.2/)

In [ ]:
%pip install numpy==1.23.5 scipy==1.9.3 pandas==1.5.2 matplotlib==3.6.2